# 1. Import Required Packages

In [ ]:
# Imports
import os
import datetime
import glob
import gdxpds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 2. Input data from User

In [ ]:
filetype_input = 'gdx' #Choose input file type: 'gdx' or 'csv'
gams_dir = 'C:/GAMS/win64/28.2' #Only required if filetype_input == 'gdx'
output='BalanceElectricityHourly'

market='FullYear' #Market analysed: 'Investment','FullYear','DayAhead','Balancing'
scenario = 'noLWnoTRAnoP2H' #Add scenario to read file name
year = 'all' #Add year to read file name (e.g. '2025', '2035', 'all')
subset = 'all' #Add subset to read file name (e.g. 'full')

first_timestep="2012-01-02"
#Meaning of SSS and TTT in the data: 'DaysHours','Hours5min','WeeksHours'
meaning_SSS_TTT='DaysHours'
#Time size of each time step in TTT for creating timestamp
size_timestep="3600s"

#scenario_names = ['noLWnoTRAnoP2H']
#market_names = ['all']
country_names = ['all']
region_names =[ 'NL']
year_names = ['2025', '2045']
season_names = ['all'] #e.g. ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007']
term_names = ['all'] #e.g. ['T001', 'S007', 'S020', 'S024', 'S028', 'S038', 'S042', 'S043']
prod_list = ['WIND', 'SOLAR-PV'] #If all production, keep empty list: []
dem_list = ['DEMAND_EXO', 'DEMAND_CCS'] #If all demand types, keep empty list: []
stackgroup = 'one' #stackgroup = 'one' for stacking, stackgroup = None for no stacking


# 3. Plot Settings

In [ ]:
# Set plotting specifications
%matplotlib inline
plt.rcParams.update({'font.size': 21})
plt.rcParams['xtick.major.pad']='12'
plt.rc('legend', fontsize=16)
y_limit = 1.1
lw = 3

# 4. Read Input Files

### 4A. CSV input

In [ ]:
if filetype_input == 'csv':
    data=pd.DataFrame()

    file = "./input/results/"+ market + "/"+ output + "_" + scenario + "_" + year + "_" + subset + ".csv"
    if os.path.isfile(file):
        df=pd.read_csv(file,encoding='utf8')
        df['Scenario'] = scenario
        df['Market']   = market
        #Renaming columns just in case timeconversion was required
        #df.rename(columns = {'G':'GGG', 'C':'CCC', 'Y':'YYY','TTT_NEW':'TTT','SSS_NEW':'SSS'}, inplace = True) 
        data=data.append(df) 


                

### 4B. GDX input

##### 4B.1 Function: reading gdx-files

In [ ]:
if filetype_input == 'gdx':
    def df_creation(gdx_file):
        df = pd.DataFrame()
        if '_' in gdx_file:
                # if yes: extract scenario name from gdx filename
            scenario = gdx_file.split('_', 3)[-3]
            year = gdx_file.split('_', 3)[-2]
            subset = gdx_file.split('_', 3)[-1][:-4]
            market = gdx_file.split('\\', 1)[0].split('/',3)[-1]
        else:
               # if no: use nan instead
            scenario = 'nan'

        # create empty temporary dataframe and load the gdx data into it
        temp = pd.DataFrame()
        temp = gdxpds.to_dataframe(gdx_file, 'EL_BALANCE_YCRST', gams_dir=gams_dir,
                               old_interface=False)

        # add a scenario column with the scenario name of the current iteration
        temp['Scenario'] = scenario
        temp['Market']  = market
        temp['run'] = scenario + '_' + year + '_' + subset

        # rearrange the columns' order
        cols = list(temp.columns)
        cols = [cols[-1]] + cols[:-1]
        temp = temp[cols]

        # concatenate the temporary dataframe to the preceeding data
        df = pd.concat([df, temp], sort=False)
        return df

##### 4B.2 Use function to read inputs

In [ ]:
if filetype_input == 'gdx':
    runs = list()
    gdx_file_list = list()
    
    gdx_file =  glob.glob('./input/results/'+ market + '\\MainResults_' + scenario + '_'  + year + '_' + subset + '.gdx')
    gdx_file = gdx_file[0]
    
    data = df_creation(gdx_file)

# 5. Add timestamps

In [ ]:
#Timestamp addition
full_timesteps = pd.read_csv('./input/full_timesteps_'+meaning_SSS_TTT+'.csv')
full_timesteps.Key=full_timesteps['SSS']+full_timesteps['TTT']
number_periods=len(full_timesteps.Key.unique())
full_timesteps['timestamp']= pd.date_range(first_timestep, periods = number_periods, freq =size_timestep)
dict_timestamp=dict(zip(full_timesteps.Key, full_timesteps.timestamp))
data['timestamp']=data['SSS']+data['TTT']
data['timestamp']=data['timestamp'].map(dict_timestamp)



# 6. Create lists

In [ ]:
#Additional set declaration
if 'C' in list(data.columns):
    data = data.rename(columns={'C':'CCC'})
ccc = list(data.CCC.unique())
rrr = list(data.RRR.unique())
elbaltype = list(data.EL_BAL_TYPE.unique())

if 'Y' in list(data.columns):
    data = data.rename(columns={'Y':'YYY'})
yyy = list(data.YYY.unique())
sss = list(full_timesteps.SSS.unique())
ttt = list(full_timesteps.TTT.unique())


# 7. Replace 'EPS'

In [ ]:
#Replace possible "Eps" with 0
if 'Value' in list(data.columns):   
    data = data.rename(columns={'Value':'Val'})
if 'Val' in list(data.columns):
    data.Val=data.Val.replace('Eps', 0)
    data.Val=pd.to_numeric(data.Val)


# 8. Plot filter selection (user selection)

In [ ]:
#Regions
if 'all' in region_names:
    r = rrr
else:
    r = region_names

#Years
if 'all' in year_names:
    y = yyy
else:
    y = year_names    
    
#Seasons    
if 'all' in season_names:
    s = sss
else:
    s = season_names

#Terms
if 'all' in term_names:
    t = ttt
else:
    t = term_names
    
data_plot=data[(data.SSS.isin(s)) & (data.TTT.isin(t)) & (data.YYY.isin(y)) & (data.RRR.isin(r))]

# 9. Prepare plotting dataframe

In [ ]:
table = pd.pivot_table(data_plot, values='Val', index=['Scenario','YYY','Market','timestamp'],
...                     columns=['EL_BAL_TYPE'], aggfunc=np.sum, fill_value=0).reset_index()
table_names=list(table.columns.values)
table.timestamp=table.timestamp.astype('datetime64[ns]')
table.sort_values(by='timestamp',inplace=True)

table.YYY = pd.to_numeric(table.YYY)

# 10. Technologies / categories selection

### 10.1 Combine columns

In [ ]:
##Combining columns (user specification)
#CHP
if 'CHP-EXTRACTION' not in table_names:
    table['CHP-EXTRACTION']=0 
if 'CHP-BACK-PRESSURE' not in table_names:
    table['CHP-BACK-PRESSURE']=0 
table['CHP']=table['CHP-BACK-PRESSURE']+table['CHP-EXTRACTION']
del table['CHP-BACK-PRESSURE']
del table['CHP-EXTRACTION'] 

#WIND
if 'WIND-OFF' not in table_names:
    table['WIND-OFF']=0 
if 'WIND-ON' not in table_names:
    table['WIND-ON']=0 
table['WIND']=table['WIND-OFF']+table['WIND-ON']
del table['WIND-ON']
del table['WIND-OFF'] 

#STORAGE PRODUCTION
if 'INTRA-STO' not in table_names:
    table['INTRA-STO']=0 
if 'INTER-STO' not in table_names:
    table['INTER-STO']=0 
table['EL-STO']=table['INTRA-STO']+table['INTER-STO']
del table['INTER-STO']
del table['INTRA-STO']

#STORAGE DEMAND
if 'DEMAND_INTERSTO' not in table_names:
    table['DEMAND_INTERSTO']=0 
if 'DEMAND_INTRA' not in table_names:
    table['DEMAND_INTRASTO']=0 
table['DEMAND_STO']=table['DEMAND_INTERSTO']+table['DEMAND_INTRASTO']
del table['DEMAND_INTERSTO']
del table['DEMAND_INTRASTO']

#INFLEX DEMAND
#if 'DEMAND_EV' not in table_names:
#    table['DEMAND_EV']=0 
#if 'DEMAND_EXO' not in table_names:
#    table['DEMAND_EXO']=0 
#table['DEMAND_INFLEX']=table['DEMAND_EV']+table['DEMAND_EXO']
#del table['DEMAND_EV']
#del table['DEMAND_EXO']

table_names=list(table.columns.values)

### 10.2 Filter production and demand

In [ ]:
all_pro=['CHP-BACK-PRESSURE', 'CHP-EXTRACTION', 'HYDRO-RESERVOIRS', 'HYDRO-RUN-OF-RIVER','HYDRO-WAVE','FUELCELL',
       'WIND-ON','WIND-OFF', 'CONDENSING', 'SOLAR-PV','INTRA-STO', 'INTER-STO','WIND','EL-STO','CHP']
if len(prod_list) > 0:
    prod_list=list(set(table_names).intersection(prod_list))
else:
    prod_list=list(set(table_names).intersection(all_pro))
    
    
    
all_dem=['DEMAND_EXO','DEMAND_P2H','DEMAND_EV','DEMAND_INTERSTO','DEMAND_INTRASTO','DEMAND_STO','DEMAND_INFLEX', 'DEMAND_OTHERTRANS',
        'DEMAND_DISTLOSSES','DEMAND_CCS','DEMAND_P2G']
if len(dem_list) > 0:
    dem_list=list(set(table_names).intersection(dem_list))
else:
    dem_list=list(set(table_names).intersection(all_dem))

# 11. Plotting settings

### 11.1 Plotting settings

In [ ]:
x=np.array(table.timestamp)

title_subplots=[]
name_subplots=[]
#for mark in market:
   # for sce in scenario:
for year in y:
    title_subplots.append(market + ' ' + scenario + ' ' + str(year))
    name_subplots.append(market + scenario + str(year))

plot_spec=[]
#for mark in market:
 #   for sce in scenario:
for year in y:
    plot_spec.append([dict({"secondary_y": True})])

legend_condition = True 


### 11.2 Add colours

In [ ]:
#Production
df_prodcolor = pd.read_excel('.\input\manual_colors_input.xlsx', sheet_name = 'Production')
df_prodcolor = df_prodcolor.dropna(axis=0)
df_prodcolor = df_prodcolor.loc[df_prodcolor['Value'].isin(prod_list),]
for prod_type in prod_list:
    if prod_type not in list(df_prodcolor.Value):
        df_prodcolor = df_prodcolor.append(pd.DataFrame(data = {'Value' : [prod_type], 'Color': px.colors.qualitative.Plotly[prod_list.index(prod_type)]}))
df_prodcolor['Value_cat'] = pd.Categorical(df_prodcolor['Value'], categories = prod_list, ordered = True )
df_prodcolor = df_prodcolor.sort_values('Value_cat')
del df_prodcolor['Value_cat']
prodcolors = list(df_prodcolor.Color)

#Demand
df_demcolor = pd.read_excel('.\input\manual_colors_input.xlsx', sheet_name = 'Demand')
df_demcolor = df_demcolor.dropna(axis = 0)
df_demcolor = df_demcolor.loc[df_demcolor['Value'].isin(dem_list), ]
for dem_type in dem_list:
    if dem_type not in list(df_demcolor.Value):
        df_demcolor = df_demcolor.append(pd.DataFrame(data = {'Value': [dem_type],'Color': px.colors.qualitative.Plotly[dem_list.index(dem_type)]}))
df_demcolor['Value_cat'] = pd.Categorical(df_demcolor['Value'], categories = dem_list, ordered = True )
df_demcolor = df_demcolor.sort_values('Value_cat')
del df_demcolor['Value_cat']
demcolors = list(df_demcolor.Color)


#Price
price_dict = pd.read_excel('.\input\manual_colors_input.xlsx', sheet_name = 'Price')
price_dict = dict(zip(list(price_dict.Value), list(price_dict.Color)))
if len(price_dict) != 1:
    price_dict = dict([('Price','pink')])


# 12 Plotting

### 12.1  Plot A: Electricity balance

In [ ]:
figA = make_subplots(rows=1*len(y)*1, cols=1,subplot_titles=title_subplots,\
                     specs=plot_spec,\
                     vertical_spacing = 0.05)
    
for year in y:
    #Production
    for i in prod_list:
        figA.add_trace(go.Scatter(x=table.loc[(table['YYY'] == int(year))].timestamp, 
                                 y=table.loc[(table['YYY'] == int(year))][i],name=i,
                                 mode = 'lines',
                                 stackgroup = stackgroup,
                                 line =  dict(color = prodcolors[prod_list.index(i)]), 
                                 showlegend= legend_condition), 
                        row= name_subplots.index(market+scenario+str(year))+1, col=1)
    #Demand    
    for i in dem_list:
        figA.add_trace(go.Scatter(x=table.loc[(table['YYY'] == int(year))].timestamp, 
                                 y=-table.loc[(table['YYY'] == int(year))][i],
                                 name=i,
                                 stackgroup = stackgroup,
                                 mode = 'lines',
                                 line =  dict(color = demcolors[dem_list.index(i)]), 
                                 showlegend= legend_condition), 
                        row = name_subplots.index(market+scenario+str(year))+1, col=1)

    figA.add_trace(
    go.Scatter(x=table.loc[(table['YYY'] == int(year))].timestamp, 
               y=table.loc[(table['YYY'] == int(year))]['PRICE'], 
               name="Price", line = dict(color = price_dict['Price'])),
    secondary_y=True,row=name_subplots.index(market+scenario+str(year))+1, col=1)  

#Layout updates
figA.update_layout(title="Electricity balance") 
figA.update_yaxes(title_text="Electricity (MWh/h)", secondary_y=False, constraintoward='middle')
figA.update_yaxes(title_text="Price (€/MWh)", secondary_y=True, constraintoward='middle')
figA.update_layout(legend_orientation="h",width=1000,height=1500)
figA.show()



### 12.2 Plot B: Electricity production

In [ ]:
figB = make_subplots(rows=1*len(y)*1, cols=1,subplot_titles=title_subplots, vertical_spacing = 0.05)

for year in y:
    #Production
    for i in prod_list:
        figB.add_trace(go.Scatter(x=table.loc[(table['YYY'] == int(year))].timestamp, 
                                 y=table.loc[(table['YYY'] == int(year))][i],name=i,
                                 mode = 'lines',
                                 stackgroup = stackgroup,
                                 line =  dict(color = prodcolors[prod_list.index(i)]), 
                                 showlegend= legend_condition), 
                        row= name_subplots.index(market+scenario+str(year))+1, col=1)



figB.update_layout(
    title="Electricity production",
)  
figB.update_yaxes(title_text="Energy (MWh/h)", secondary_y=False)
figB.update_layout(legend_orientation="h",width=1000,height=1500)
figB.show()


### 12.3 Plot C: Electricity demand

In [ ]:
figC = make_subplots(rows=1*len(y)*1, cols=1,subplot_titles=title_subplots, vertical_spacing = 0.05)


for year in y:
    #Demand    
    for i in dem_list:
        figC.add_trace(go.Scatter(x=table.loc[(table['YYY'] == int(year))].timestamp, 
                                 y=table.loc[(table['YYY'] == int(year))][i],name=i,
                                 mode = 'lines',
                                 stackgroup = stackgroup,
                                 line =  dict(color = demcolors[dem_list.index(i)]), 
                                 showlegend= legend_condition), 
                        row = name_subplots.index(market+scenario+str(year))+1, col=1)




figC.update_layout(title="Electricity demand",)
figC.update_yaxes(title_text="Energy (MWh/h)", secondary_y=False)
figC.update_layout(legend_orientation="h",width=1000,height=1500)
figC.show()

# X - Export

##### X.1 - Make output directories


In [ ]:
# Make output folder
if not os.path.isdir('output'):
    os.makedirs('output')

In [ ]:
# Make output folder
if not os.path.isdir('output/' + output):
    os.makedirs('output/' + output)

##### X.2 - Create HTML outputs

In [ ]:
figA.write_html('.\output/' + output + '//ElectricityBalance_'+str(scenario)[2:-2] + '_' + str(year)+'.html')
figB.write_html('.\output/' + output + '//ElectricityProduction_'+str(scenario)[2:-2] + '_' + str(year)+'.html')
figC.write_html('.\output/' + output + '//ElectricityDemand_'+str(scenario)[2:-2] + '_' + str(year)+'.html')
